# ML4NLP1
## Starting Point for Exercise 1, part II

This notebook is supposed to serve as a starting point and/or inspiration when starting exercise 1, part II.

One of the goals of this exercise is o make you acquainted with **skorch**. You will probably need to consult the [documentation](https://skorch.readthedocs.io/en/stable/).

# Installing skorch and loading libraries

In [ ]:
import subprocess

# Installation on Google Colab
try:
    import google.colab
    subprocess.run(['python', '-m', 'pip', 'install', 'skorch'])
except ImportError:
    pass

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
from skorch import NeuralNetClassifier

In [ ]:
torch.manual_seed(0)
torch.cuda.manual_seed(0)

In [ ]:
import pandas as pd
import numpy as np
import csv
import re
import string
from collections import defaultdict

## Training a classifier and making predictions

In [ ]:
# download dataset
!gdown 1QP6YuwdKFNUPpvhOaAcvv2Pcp4JMbIRs # x_train
!gdown 1QVo7PZAdiZKzifK8kwhEr_umosiDCUx6 # x_test
!gdown 1QbBeKcmG2ZyAEFB3AKGTgSWQ1YEMn2jl # y_train
!gdown 1QaZj6bI7_78ymnN8IpSk4gVvg-C9fA6X # y_test

Downloading...
From: https://drive.google.com/uc?id=1QP6YuwdKFNUPpvhOaAcvv2Pcp4JMbIRs
To: /content/x_train.txt
100% 64.1M/64.1M [00:01<00:00, 41.6MB/s]
Downloading...
From: https://drive.google.com/uc?id=1QVo7PZAdiZKzifK8kwhEr_umosiDCUx6
To: /content/x_test.txt
100% 65.2M/65.2M [00:00<00:00, 70.5MB/s]
Downloading...
From: https://drive.google.com/uc?id=1QbBeKcmG2ZyAEFB3AKGTgSWQ1YEMn2jl
To: /content/y_train.txt
100% 480k/480k [00:00<00:00, 103MB/s]
Downloading...
From: https://drive.google.com/uc?id=1QaZj6bI7_78ymnN8IpSk4gVvg-C9fA6X
To: /content/y_test.txt
100% 480k/480k [00:00<00:00, 128MB/s]


In [ ]:
with open(f'x_train.txt') as f:
    x_train = f.read().splitlines()
with open(f'y_train.txt') as f:
    y_train = f.read().splitlines()
with open(f'x_test.txt') as f:
    x_test = f.read().splitlines()
with open(f'y_test.txt') as f:
    y_test = f.read().splitlines()

In [ ]:
import pandas as pd
# combine x_train and y_train into one dataframe
train_df = pd.DataFrame({'text': x_train, 'label': y_train})

#combine x_test and y_test into one dataframe
test_df = pd.DataFrame({'text': x_test, 'label': y_test})

In [ ]:
# T: Please use again the train/test data that includes English, German, Dutch, Danish, Swedish and Norwegian, plus 20 additional languages of your choice (the labels can be found in the file labels.csv)
# and adjust the train/test split if needed


import pandas as pd
from sklearn.model_selection import train_test_split
test_df_to_train, test_df_remaining = train_test_split(test_df, test_size=0.5, random_state=42, stratify=test_df['label'])

train_df = pd.concat([train_df, test_df_to_train], ignore_index=True)

test_df = test_df_remaining

fixed_languages = ['eng', 'deu', 'nld', 'dan', 'swe', 'nor']
self_selected_languages = ['est', 'tha', 'guj', 'tam', 'vie', 'lat', 'urd', 'por', 'fra', 'rus', 'ara', 'heb', 'hin', 'jpn', 'kor', 'zho', 'spa', 'ita', 'tur', 'ell']
final_language_list = fixed_languages + self_selected_languages
sub_traindf = train_df[train_df['label'].isin(final_language_list)]
sub_testdf = test_df[test_df['label'].isin(final_language_list)]


In [ ]:
# T: use your adjusted code to encode the labels here
# T: With the following code, we wanted to encode the labels, however, our cat was walking on the keyboard and some of it got changed. Can you fix it?
from sklearn.preprocessing import LabelEncoder
le_fitted = LabelEncoder().fit(sub_traindf['label'])
y_train_dev, y_test = le_fitted.transform(sub_traindf['label']), le_fitted.transform(sub_testdf['label'])


In [ ]:
# T: In the following, you can find a small (almost) working example of a neural network. Unfortunately, again, the cat messed up some of the code. Please fix the code such that it is executable.

In [ ]:
# First, we extract some simple features as input for the neural network
# import package:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(analyzer='char', ngram_range=(2, 2), max_features=1000, binary=True)
X = vectorizer.fit_transform(sub_traindf['text'].to_numpy())

In [ ]:
X = X.astype(np.float32)
y = y_train_dev.astype(np.int64)

In the following, we define a vanilla neural network with two hidden layers. The output layer should have as many outputs as there are classes. In addition, it should have a nonlinearity function.

In [ ]:
class ClassifierModule(nn.Module):
    def __init__(
            self,
            num_units=200,
            nonlin=F.relu,
    ):
        super(ClassifierModule, self).__init__()
        self.num_units = num_units
        self.nonlin = nonlin

        self.dense0 = nn.Linear(1000, num_units)
        self.nonlin = nonlin
        self.dense1 = nn.Linear(num_units, 50)
        self.output = nn.Linear(50, 25)

    def forward(self, X, **kwargs):
      X = self.nonlin(self.dense0(X))
      X = F.relu(self.dense1(X))
      X = self.output(X)
      return X.squeeze(dim=1)

In [ ]:

from sklearn.model_selection import GridSearchCV
from skorch.callbacks import EarlyStopping
net = NeuralNetClassifier(
    ClassifierModule,
    max_epochs=20,
    criterion=nn.CrossEntropyLoss(),
    callbacks=[('earlystopping', EarlyStopping(patience=5))],
    lr=0.1,
    #device='cuda',  # comment this to train with CPU
)
params = {
    'optimizer': [torch.optim.SGD, torch.optim.Adam],
    'module__nonlin':[F.relu, F.tanh],
    'module__num_units': [800,200],
    'callbacks__earlystopping__patience':[5,10]
}

# Note: Consider using StratifiedKFold for classification tasks
gs = GridSearchCV(net, params, refit=True, cv=3, scoring='accuracy', verbose=3)

In [ ]:
print(X.shape)
print(y_train_dev.shape)

(18750, 1000)
(18750,)


In [ ]:
gs.fit(X, y_train_dev)

Fitting 3 folds for each of 16 candidates, totalling 48 fits
  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        2.8769       0.4356        2.4801  2.2969
      2        1.4343       0.5916        1.3274  2.2242
      3        0.5960       0.8956        0.4880  2.3504
      4        0.3404       0.9496        0.3138  2.9736
      5        0.2463       0.9556        0.2495  2.1917
      6        0.1972       0.9592        0.2166  2.1699
      7        0.1673       0.9604        0.1973  2.2055
      8        0.1465       0.9616        0.1844  2.1756
      9        0.1302       0.9624        0.1747  3.2393
     10        0.1164       0.9640        0.1669  2.1843
     11        0.1044       0.9648        0.1607  2.2160
     12        0.0940       0.9656        0.1558  2.1917
     13        0.0850       0.9652        0.1523  2.1812
     14        0.0774       0.9648        0.1498  2.7670
     15        0.0708      

GridSearchCV(cv=3,
             estimator=<class 'skorch.classifier.NeuralNetClassifier'>[uninitialized](
  module=<class '__main__.ClassifierModule'>,
),
             param_grid={'callbacks__earlystopping__patience': [5, 10],
                         'module__nonlin': [<function relu at 0x7ffa9858a560>,
                                            <function tanh at 0x7ffa9858af80>],
                         'module__num_units': [800, 200],
                         'optimizer': [<class 'torch.optim.sgd.SGD'>,
                                       <class 'torch.optim.adam.Adam'>]},
             scoring='accuracy', verbose=3)

In [ ]:
from sklearn.metrics import classification_report
# First, we extract some simple features as input for the neural network
# import package:
from sklearn.feature_extraction.text import CountVectorizer

X_test = vectorizer.transform(sub_testdf['text'].to_numpy())

X_test = X_test.astype(np.float32)
y_test = y_test.astype(np.int64)
y_pred = gs.predict(X_test)

print(classification_report(y_test , y_pred))

              precision    recall  f1-score   support

           0       1.00      0.99      0.99       250
           1       0.97      0.97      0.97       250
           2       0.94      0.96      0.95       250
           3       1.00      0.98      0.99       250
           4       0.86      0.97      0.91       250
           5       0.98      0.97      0.97       250
           6       0.97      0.98      0.98       250
           7       0.99      0.97      0.98       250
           8       1.00      1.00      1.00       250
           9       1.00      0.98      0.99       250
          10       0.96      0.98      0.97       250
          11       0.98      0.76      0.86       250
          12       1.00      0.98      0.99       250
          13       0.95      0.96      0.96       250
          14       0.98      0.99      0.99       250
          15       0.99      0.94      0.97       250
          16       0.98      1.00      0.99       250
          17       0.96    

In [ ]:
# max_feature is 100 here(ablation)
class ClassifierModule(nn.Module):
    def __init__(
            self,
            num_units=200,
            nonlin=F.relu,
    ):
        super(ClassifierModule, self).__init__()
        self.num_units = num_units
        self.nonlin = nonlin

        self.dense0 = nn.Linear(100, num_units)
        self.nonlin = nonlin
        self.dense1 = nn.Linear(num_units, 50)
        self.output = nn.Linear(50, 25)

    def forward(self, X, **kwargs):
      X = self.nonlin(self.dense0(X))
      X = F.relu(self.dense1(X))
      X = self.output(X)
      return X.squeeze(dim=1)

net2 = NeuralNetClassifier(
    ClassifierModule,
    max_epochs=20,
    criterion=nn.CrossEntropyLoss(),
    callbacks=[('earlystopping', EarlyStopping(patience=5))],
    lr=0.1,
    #device='cuda',  # comment this to train with CPU
)
params2 = {
    'optimizer': [torch.optim.SGD, torch.optim.Adam],
    'module__nonlin':[F.relu, F.tanh],
    'module__num_units': [800,200],
    'callbacks__earlystopping__patience':[5,10]
}
vectorizer2 = CountVectorizer(analyzer='char', ngram_range=(2, 2), max_features=100, binary=True)
X_100 = vectorizer2.fit_transform(sub_traindf['text'].to_numpy())
X_100 = X_100.astype(np.float32)
y = y_train_dev.astype(np.int64)
# Note: Consider using StratifiedKFold for classification tasks
gs2 = GridSearchCV(net2, params2, refit=True, cv=3, scoring='accuracy', verbose=3)
gs2.fit(X_100, y_train_dev)

Fitting 3 folds for each of 16 candidates, totalling 48 fits
  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        2.9187       0.1332        2.8896  2.0389
      2        2.3096       0.2028        2.3778  1.5750
      3        1.8023       0.2696        1.9711  1.6166
      4        1.5523       0.3812        1.7371  1.5557
      5        1.3838       0.4268        1.6145  1.5697
      6        1.2665       0.4736        1.5108  1.5597
      7        1.1934       0.4984        1.4275  1.8590
      8        1.1447       0.5152        1.3647  2.3187
      9        1.1098       0.5428        1.3164  1.5771
     10        1.0829       0.5520        1.2757  1.5797
     11        1.0613       0.5652        1.2418  1.6053
     12        1.0436       0.5712        1.2136  1.6046
     13        1.0285       0.5788        1.1912  1.6038
     14        1.0158       0.5820        1.1733  1.5447
     15        1.0047      

GridSearchCV(cv=3,
             estimator=<class 'skorch.classifier.NeuralNetClassifier'>[uninitialized](
  module=<class '__main__.ClassifierModule'>,
),
             param_grid={'callbacks__earlystopping__patience': [5, 10],
                         'module__nonlin': [<function relu at 0x7ffa9858a560>,
                                            <function tanh at 0x7ffa9858af80>],
                         'module__num_units': [800, 200],
                         'optimizer': [<class 'torch.optim.sgd.SGD'>,
                                       <class 'torch.optim.adam.Adam'>]},
             scoring='accuracy', verbose=3)

In [ ]:
from sklearn.metrics import classification_report
# First, we extract some simple features as input for the neural network
# import package:
from sklearn.feature_extraction.text import CountVectorizer

X_test_2 = vectorizer2.transform(sub_testdf['text'].to_numpy())

X_test_2 = X_test_2.astype(np.float32)
y_test = y_test.astype(np.int64)
y_pred = gs2.predict(X_test_2)

print(classification_report(y_test , y_pred))

              precision    recall  f1-score   support

           0       0.30      0.46      0.36       250
           1       0.90      0.80      0.84       250
           2       0.92      0.78      0.85       250
           3       0.32      0.05      0.09       250
           4       0.82      0.83      0.82       250
           5       0.77      0.96      0.86       250
           6       0.90      0.91      0.90       250
           7       0.45      0.29      0.35       250
           8       0.26      0.06      0.10       250
           9       0.43      0.49      0.46       250
          10       0.92      0.91      0.91       250
          11       0.45      0.04      0.07       250
          12       0.29      0.62      0.39       250
          13       0.83      0.86      0.85       250
          14       0.82      0.91      0.86       250
          15       0.91      0.85      0.88       250
          16       0.24      0.35      0.29       250
          17       0.88    

In [ ]:
best_parameters = gs.best_params_


In [ ]:
print(best_parameters)

{'callbacks__earlystopping__patience': 10, 'module__nonlin': <function tanh at 0x7ffa9858af80>, 'module__num_units': 800, 'optimizer': <class 'torch.optim.sgd.SGD'>}
